In [1]:
import sys,re,os
import datetime,time
# from pyspark import SparkContext
# from pyspark import SparkConf
from operator import add
import json,math,random
import base64
reload(sys)
sys.setdefaultencoding( "utf-8" )

In [2]:
day = '2018-04-10'

In [3]:
format_day = day.replace('-','')
month = format_day[0:6]
month

'201804'

In [4]:
yesterday = '2018-04-09'.replace('-','')

In [5]:
click_path = '/user/portal/ODM/SDK/ODM_SDK_MOBILE_LOG/month=%s/day=%s/*/*' %(month,format_day)
today_topic_path = '/user/datacenter/data/headline_ctr/doc_extract4/month=%s/day=%s/*/*.gz,/user/datacenter/data/openplat_ctr/doc_extract/month=%s/day=%s/*/*.gz' %(month,format_day,month,format_day)
old_topic_path = '/user/datacenter/linguangjie/user-profile-history/headline_topic_backup/' + yesterday
video_topic_path = '/user/datacenter/liyupeng/liujian/item_info_merge/final'
new_topic_path = '/user/datacenter/linguangjie/user-profile-history/headline_topic_backup/' + format_day
old_profile_path = 'hdfs://hz-cluster3/user/datacenter/linguangjie/user-profile-history/all-profile/%s/*' %(yesterday).replace('-','')


In [6]:
sc.addPyFile("hdfs://hz-cluster3/user/datacenter/linguangjie/bin/Crypto.zip")
sc.addPyFile("hdfs://hz-cluster3/user/datacenter/linguangjie/bin/rediscluster.zip")
sc.addPyFile("hdfs://hz-cluster3/user/datacenter/linguangjie/bin/common-gen-py.zip")
sc.addPyFile("hdfs://hz-cluster3/user/datacenter/linguangjie/bin/ufs.zip")
sc.addPyFile("hdfs://hz-cluster3/user/datacenter/linguangjie/bin/mmhash.so")

In [7]:
today_topic_line = sc.textFile(today_topic_path)
old_topic_line =  sc.textFile(old_topic_path)
video_topic_line = sc.textFile(video_topic_path)

In [8]:
line = today_topic_line.first()

In [9]:
topic_json = json.loads(line)

In [10]:
topic_json['docid']

u'DEUGRSMI0517TE39'

In [11]:
topic_json['expireTime']

u'2018-04-12 08:11:00'

In [12]:
def device_id_parse(enStr, key):
    from Crypto.Cipher import AES
    cipher = AES.new(key, AES.MODE_ECB)
    enStr += (len(enStr) % 4)*"="
    decryptByts = base64.b64decode(enStr)
    msg = cipher.decrypt(decryptByts)
    paddingLen = ord(msg[len(msg)-1])
    return msg[0:-paddingLen]

def parse_userid_docid(click_data):
    body_str =click_data.split('message:')
    try:
        message = json.loads(body_str[1])
        for item in message:
            if "u" not in item or 'e' not in item:
                continue
            if "id" in item and item['id'] in ('91XRY1','lN09e3','RMN9XR'):
                uid_type = 'tsb'
            elif "id" in item and item['id'] in ('TV51dh'):
                uid_type = 'kfpt'
            else:
                continue

            uuid = item['u']
            for arrays in item['e']:
                if  'n' in arrays and 'ts' in arrays and arrays['n'] in ["_vvX","_pvX","_svX","_ivX"] and \
                    'du' in arrays and 'kv' in arrays and 'id' in arrays['kv'] and 'column' in arrays['kv'] :
                    occur_ts = str(arrays['ts'])
                    du = str(arrays['du'])
                    docid = str(arrays['kv']['id'])
                    if ( not docid.startswith('V')) and ( not (len(docid) == 16 and docid[8:12].isdigit())):
                        continue
                    offset = ""
                    sessionid = ""
                    passport = ""
                    if 'p' in arrays:
                        passport = device_id_parse(arrays['p'],'neteasemobiledat')
                    
                    if passport != "":
                        yield (docid,passport)
                    yield (docid,uuid)                        
    except:
        pass

# Click Data

In [13]:
click_line = sc.textFile(click_path)

In [14]:
click_data = click_line.flatMap(parse_userid_docid)

In [15]:
click_data.take(10)

[('DEQT9MKL0522AH2L', u'CQkyOGEzY2ZkNTgwOTk4YWY4CTYzMzUwMDE%3D'),
 ('VIDSP4K8V', u'CQkyOGEzY2ZkNTgwOTk4YWY4CTYzMzUwMDE%3D'),
 ('VHDI3SDBC', u'CQkyOGEzY2ZkNTgwOTk4YWY4CTYzMzUwMDE%3D'),
 ('DEQUSOBL0522A15A', u'CQkyOGEzY2ZkNTgwOTk4YWY4CTYzMzUwMDE%3D'),
 ('VNCSP1LI8', u'CQljMDMxZjExYmVmMDBiNTNiCVoyWDRDMTVCMDMwMTQ2MDc%3D'),
 ('DEV6NNJJ0528QQAC', u'CQkyMWU4MGQwNmZkY2NkYTQwCUdKUTdOMTgzMDkwMDY4OTE%3D'),
 ('D3ND3HE00517A6VE', u'CQllYmMxMDMyMWQ2ZDVhMGY3CUxFNjdBMDYxODAzMDEzMjU%3D'),
 ('DE2Q8VLB0526PHHI', u'866820010264095'),
 ('VADQM8PMH', u'CQk2MDZhYjYwMTUyZmEyY2I1CURCSDlYQTE3NDIyMDEwOTI%3D'),
 ('VRDFBKS0U', u'CQk4ZDQ3ZWNjNGIxYjhjODE3CTQ3MzNlMWVl')]

# profile data

In [38]:
old_profile_path = '/user/datacenter/linguangjie/user-profile-history/all-profile/20180407/*'

In [39]:
old_profile_line = sc.textFile(old_profile_path)

In [42]:
lines = old_profile_line.collect()

In [43]:
len(lines)

5106

In [47]:
def parse_old_profile(line):
    try:
        item = eval(line.encode('utf-8'))
    except:
        return
    
    if len(item) == 3:
        uid = item[0]
        topic_dic = item[1]
        day_tag = int(item[2])
        if uid != "" and  uid != "-" and len(uid) > 6:
            return (uid,('old', topic_dic, day_tag))

In [48]:
profiles = filter(None, map(parse_old_profile, lines))

# calc profile data

# parse_docic_feature

In [63]:
TOPIC_VERSION = '0000 0000 0000 0000 0010 0001'
VEDIO_TITLE_KW_V2_PROFILE_VERSION = '0011 0000 0000 0001 1100 0001'
VEDIO_MANUALLY_CATEGORY_PROFILE_VERSION = '0011 0000 0000 0001 0010 0001'
VEDIO_MANUALLY_CATEGORY1_PROFILE_VERSION = '0011 0000 0000 0001 1010 0001'
VEDIO_EDIT_KW_PROFILE_VERSION =  '0011 0000 0000 0000 1100 0001'
VEDIO_INTEREST_PROFILE_VERSION = '0011 0000 0000 0001 0100 0001'
HEADLINE_TITLE_KW_PROFILE_VERSION = '0000 0000 0000 0001 1100 0001'
HEADLINE_DOC_KW_PROFILE_VERSION =   '0000 0000 0000 0000 0101 0001'
HEADLINE_EDIT_KW_PROFILE_VERSION =  '0000 0000 0000 0000 1100 0001'
HEADLINE_INTEREST_PROFILE_VERSION = '0000 0000 0000 0001 0100 0001'
HEADLINE_MANUALLY_CATEGORY_PROFILE_VERSION = '0000 0000 0000 0001 0010 0001'
HEADLINE_LEVEL_CATEGORY_PROFILE_VERSION =    '0000 0000 0000 0001 1010 0001'
HEADLINE_W2V_PROFILE_VERSION =  '0000 0000 0000 0000 0010 0001' 

In [64]:
video_title_kw_v2_field = str(hex(int(VEDIO_TITLE_KW_V2_PROFILE_VERSION.replace(' ',""),2)))[2:].upper()
video_title_kw_v2_field

'3001C1'

In [65]:
video_manually_category_field = str(hex(int(VEDIO_MANUALLY_CATEGORY_PROFILE_VERSION.replace(' ',""),2)))[2:].upper()
video_manually_category_field

'300121'

In [66]:
video_manually_category1_field =  str(hex(int(VEDIO_MANUALLY_CATEGORY1_PROFILE_VERSION.replace(' ',""),2)))[2:].upper()
video_manually_category1_field

'3001A1'

In [67]:
video_edit_kw_field  = str(hex(int(VEDIO_EDIT_KW_PROFILE_VERSION.replace(' ',""),2)))[2:].upper()
video_interest_field = str(hex(int(VEDIO_INTEREST_PROFILE_VERSION.replace(' ',""),2)))[2:].upper()
headline_title_kw_field = str(hex(int(HEADLINE_TITLE_KW_PROFILE_VERSION.replace(' ',""),2)))[2:].upper()
headline_doc_kw_field = str(hex(int(HEADLINE_DOC_KW_PROFILE_VERSION.replace(' ',""),2)))[2:].upper()
headline_edit_kw_field  = str(hex(int(HEADLINE_EDIT_KW_PROFILE_VERSION.replace(' ',""),2)))[2:].upper()
headline_interest_field = str(hex(int(HEADLINE_INTEREST_PROFILE_VERSION.replace(' ',""),2)))[2:].upper()
headline_manually_category_field = str(hex(int(HEADLINE_MANUALLY_CATEGORY_PROFILE_VERSION.replace(' ',""),2)))[2:].upper()
headline_level_category_field = str(hex(int(HEADLINE_LEVEL_CATEGORY_PROFILE_VERSION.replace(' ',""),2)))[2:].upper()
headline_w2v_field = str(hex(int(HEADLINE_W2V_PROFILE_VERSION.replace(' ',""),2)))[2:].upper()

In [70]:
headline_level_category_field

'1A1'

In [71]:
feature_json['title_kw_v2']

NameError: name 'feature_json' is not defined

In [78]:
type(today_topic_line)

pyspark.rdd.RDD

In [82]:
topic_json.keys()

[u'img_embedding',
 u'title_byte_count',
 u'attitudesCount',
 u'playlength',
 u'imgType',
 u'doc_word_count',
 u'digest',
 u'title',
 u'source',
 u'expireTime2',
 u'kwList',
 u'repostsCount',
 u'video_origin',
 u'receive_time',
 u'dkeys',
 u'photosetID',
 u'imgsrc',
 u'skipType',
 u'data_source',
 u'imgnewextra',
 u'bodySegObj',
 u'sectionsid',
 u'sansu_score',
 u'docUrl',
 u'end_time',
 u'title_kw',
 u'largeurl',
 u'localcity',
 u'imgsum',
 u'clusterId',
 u'mp4SdUrl',
 u'create_time',
 u'apollo_docid',
 u'topic_w2v',
 u'channelType',
 u'category',
 u'docid',
 u'doc_segment',
 u'doc_byte_count',
 u'title_char_count',
 u'origion',
 u'sourceLevel',
 u'TAGS',
 u'segSimHashObj',
 u'title_word_count',
 u'publisher',
 u'titlekwList_v2',
 u'sizeSHD',
 u'lmodify',
 u'coverUrls',
 u'sectiondescription',
 u'playersize',
 u'sizeSD',
 u'vid',
 u'sizeHD',
 u'accountCategory',
 u'quality',
 u'category_v3',
 u'sectiontitle',
 u'vulgar',
 u'titleCheat',
 u'is_santu',
 u'body_limit_300',
 u'multiurl',


In [91]:
from __future__ import unicode_literals

In [100]:
topic_json['title']

u'\u53c8\u4e00\u90e8\u5927\u5236\u4f5c\uff1f\u516d\u5c0f\u9f84\u7ae5\u4e3b\u6f14\u5f71\u89c6\u7248\u897f\u6e38\u8bb0\uff0c\u8fd9\u624d\u662f\u6b63\u5b97\u7684\u7f8e\u7334\u738b\uff01'

In [102]:
topic_json['title'].encode("UTF-8")

'\xe5\x8f\x88\xe4\xb8\x80\xe9\x83\xa8\xe5\xa4\xa7\xe5\x88\xb6\xe4\xbd\x9c\xef\xbc\x9f\xe5\x85\xad\xe5\xb0\x8f\xe9\xbe\x84\xe7\xab\xa5\xe4\xb8\xbb\xe6\xbc\x94\xe5\xbd\xb1\xe8\xa7\x86\xe7\x89\x88\xe8\xa5\xbf\xe6\xb8\xb8\xe8\xae\xb0\xef\xbc\x8c\xe8\xbf\x99\xe6\x89\x8d\xe6\x98\xaf\xe6\xad\xa3\xe5\xae\x97\xe7\x9a\x84\xe7\xbe\x8e\xe7\x8c\xb4\xe7\x8e\x8b\xef\xbc\x81'

In [110]:
print 123

In [137]:
import pprint as pp

In [138]:
pp.pprint(topic_json['kwList'])

In [127]:
eval(line)

NameError: name 'false' is not defined

In [134]:
#coding=utf-8 
a = u'你好'

In [142]:
a

u'\u4f60\u597d'

In [21]:
print a

NameError: name 'a' is not defined

In [27]:
'ab' in ('abcd', )

False

In [29]:
('abcd')

str

In [ ]:
()